# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
# loading required libraries and starting spark session
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 
from pyspark.sql import Window

import json

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 17:19:26,842 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
# read the data

schema = StructType([StructField('user_id', IntegerType(), True), 
                     StructField('session_id', IntegerType(), True),
                     StructField('event_type', StringType(), True),
                     StructField('event_page', StringType(), True),
                     StructField('timestamp', IntegerType(), True)]) 


clickstream = se.read.csv("hdfs:/data/clickstream.csv", header=True, sep = '\t', schema = schema)

clickstream.registerTempTable("clickstream")
clickstream.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

# Spark SQL

In [3]:
# sql solution 
result_sql = se.sql("""
select route, count(*) as freq
from
    (select user_id, session_id, concat_ws('-',collect_list(event_page)) route
    from
        (select clickstream.user_id, clickstream.session_id, clickstream.event_page, lag(clickstream.event_page, 1, '-1') over(partition by clickstream.user_id, clickstream.session_id order by clickstream.timestamp) prev_page
            from clickstream 
                left join (select user_id, session_id, min(timestamp) timestamp
                        from clickstream
                        where event_type like '%error%' 
                        group by user_id, session_id) as err on clickstream.user_id = err.user_id and clickstream.session_id = err.session_id
            where clickstream.timestamp < err.timestamp or err.timestamp is null) 
    where event_page <> prev_page
    group by user_id, session_id)
group by route
order by freq desc
limit 30
""")

result_sql.show(n = 30)

+--------------------+----+
|               route|freq|
+--------------------+----+
|                main|8184|
|        main-archive|1112|
|         main-rabota|1046|
|       main-internet| 897|
|          main-bonus| 870|
|           main-news| 769|
|        main-tariffs| 676|
|         main-online| 587|
|          main-vklad| 518|
| main-rabota-archive| 170|
| main-archive-rabota| 167|
|  main-bonus-archive| 143|
|   main-rabota-bonus| 139|
|   main-bonus-rabota| 135|
|    main-news-rabota| 135|
|main-archive-inte...| 132|
|main-internet-rabota| 129|
|    main-rabota-news| 129|
|   main-archive-news| 126|
|main-rabota-internet| 124|
|main-internet-arc...| 123|
|  main-archive-bonus| 117|
| main-internet-bonus| 115|
|main-tariffs-inte...| 114|
|   main-news-archive| 113|
|  main-news-internet| 109|
|main-archive-tariffs| 104|
|  main-internet-news| 103|
|main-tariffs-archive| 103|
|    main-rabota-main|  94|
+--------------------+----+



In [4]:
# creating the file with results for spark SQL, as described in the task
with open("results_sql.txt", "w") as f:
    for row in result_sql.collect():
        res_row = row['route'] + '\t' + str(row['freq']) + '\n'
        f.write(res_row)

# Spark RDD

In [5]:
# rdd solution
# creating a dictionay of first errors per user session
err = (
    clickstream.rdd
    .filter(lambda x: x.event_type != 'event' and x.event_type != 'page' )
    .map(lambda x: (str(x.user_id)+str(x.session_id), x.timestamp))
    .reduceByKey(lambda a, b: min(a,b))
    .collect()
)
err = {i: t for i, t in err}

In [6]:
# defining the functions we will need

def filter_err (x, err):
    err_ind = err.setdefault(x[0], 'no_error')
    if err_ind == 'no_error':
        return True
    elif err_ind > x[2]:
        return True
    else:
        return False

def get_route(a, b):
    prev = a.split('-')[-1]
    if prev != b:
        return a+'-'+b
    else:
        return a
    

In [7]:
result_rdd = (
    clickstream.rdd
    .map(lambda x: (str(x.user_id)+str(x.session_id), x.event_page, x.timestamp))
    .filter(lambda x:  filter_err (x, err))
    .map(lambda x: (x[0], x[1]))
    .reduceByKey(lambda a, b: get_route(a, b))
    .map(lambda x: (x[1], 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(keyfunc = lambda x: x[1], ascending=False)
    .take(30)
)

result_rdd

[('main', 8178),
 ('main-archive', 1098),
 ('main-rabota', 1038),
 ('main-internet', 884),
 ('main-bonus', 864),
 ('main-news', 760),
 ('main-tariffs', 670),
 ('main-online', 585),
 ('main-vklad', 513),
 ('main-rabota-archive', 168),
 ('main-archive-rabota', 167),
 ('main-rabota-bonus', 139),
 ('main-bonus-archive', 139),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 134),
 ('main-archive-internet', 132),
 ('main-rabota-news', 128),
 ('main-internet-rabota', 128),
 ('main-archive-news', 124),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-tariffs-internet', 114),
 ('main-internet-bonus', 114),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-tariffs-archive', 102),
 ('main-internet-news', 101),
 ('main-rabota-main', 95)]

In [8]:
# creating the file with results for spark RDD
with open("results_rdd.txt", "w") as f:
    for row in result_rdd:
        res_row = row[0] + '\t' + str(row[1]) + '\n'
        f.write(res_row)

# Spark DF

In [9]:
# df solution
# creating a df of first errors per user session
error_times = (
    clickstream
    .filter('event_type not in ("page", "event")')
    .groupBy(['user_id', 'session_id'])
    .agg(F.min('timestamp').alias('ts'))
    .select(F.col('user_id').alias('u'), F.col('session_id').alias('s'), 'ts')
)

In [10]:
w = Window.partitionBy(['user_id', 'session_id']).orderBy('timestamp')

result_df = (
    clickstream
    .join(error_times, [clickstream.user_id ==  error_times.u, clickstream.session_id ==  error_times.s], 'left')
    .filter('timestamp < ts or ts is null')
    .withColumn('prev', F.lag('event_page', 1, '-1').over(w))
    .filter('event_page <> prev')
    .groupBy(['user_id', 'session_id']).agg(F.array_join(F.collect_list('event_page'),'-').alias('route'))
    .groupBy('route').count()
    .orderBy(F.desc('count'))
    .limit(30)
)


result_df.show(n = 30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1112|
|         main-rabota| 1046|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  676|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|main-internet-rabota|  129|
|    main-rabota-news|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [11]:
# creating the file with results for spark DF
with open("results_df.txt", "w") as f:
    for row in result_df.collect():
        res_row = row['route'] + '\t' + str(row['count']) + '\n'
        f.write(res_row)

In [12]:
# stopping spark
sc.stop()